<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-04 10:29:22
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.37 C
-------------------
Today PnL: 34.40 K (0.25%)
Current PnL: -13.03 L (-9.08%)
CY Booked + Current PnL: -6.29 L (-4.38%)
-------------------
Total profit:  4.80 L
Total loss:  -17.83 L
-------------------
Total Booked + Current PnL: 20.69 L (17.76%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.92%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.44 L (52.19%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,1.67,M-LC,34.38,101970.0,23608.0,10421.0,0.36,30.13,10.22,43.43,77.0,2.27,0.75,37.05,XY24,NTT,MISC
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,93.75,132408.0,-10071.0,10699.0,-0.16,-7.07,8.08,0.44,243.0,-0.94,0.97,56.28,OX40N,NTT,BANKS
5,APOLLOHOSP,6904.43,8285.00,-16.74,H-LC,87.50,113490.0,9924.0,10782.0,0.01,9.58,9.50,20.00,30.0,0.92,0.83,25.00,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-19.90,H-LC,82.29,110608.0,6296.0,11238.0,-0.22,6.04,10.16,16.81,27.0,0.56,0.81,23.05,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,0.15,M-LC,30.21,173495.0,32655.0,11832.0,0.66,23.19,6.82,31.59,89.0,2.76,1.27,50.71,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,19.79,181546.0,1738.0,110670.0,-0.38,0.97,60.96,62.52,171.0,0.02,1.33,35.38,XR,NTT,BANKS
55,LALPATHLAB,2804.19,3545.00,-32.87,H-SC,23.96,154440.0,210.0,40540.0,1.86,0.14,26.25,26.42,125.0,0.01,1.13,21.79,X40N,NTT,HEALTHCARE
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,57.29,151970.0,1057.0,59390.0,0.73,0.70,39.08,40.06,102.0,0.02,1.11,24.30,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,12.50,49150.0,7895.0,2910.0,5.10,19.14,5.92,26.19,1.0,2.71,0.36,15.72,XY24,ATH,IT
65,RELAXO,902.64,1176.00,-35.80,H-SC,58.33,58016.0,-57522.0,92512.0,3.58,-49.79,159.46,30.28,134.0,-0.62,0.42,13.34,X40N,NTT,FOOTWEAR
70,SATIN,199.77,274.00,-15.90,H-SC,86.46,166868.0,-29106.0,101923.0,2.18,-14.85,61.08,37.16,130.0,-0.29,1.22,24.62,XY24,NTT,FINANCE
78,SURYODAY,189.78,240.00,57.89,H-SC,10.42,111753.0,-38173.0,77847.0,2.11,-25.46,69.66,26.46,145.0,-0.49,0.82,43.16,XR,NTT,BANKS
55,LALPATHLAB,2804.19,3545.00,-32.87,H-SC,23.96,154440.0,210.0,40540.0,1.86,0.14,26.25,26.42,125.0,0.01,1.13,21.79,X40N,NTT,HEALTHCARE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SAMMAANCAP,170.35,326.00,-164.77,M-SC,41.67,74970.0,-27240.0,120627.0,-7.94,-26.65,160.90,91.37,206.0,-0.23,0.55,21.70,XY25,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-29.71,H-SC,75.00,210250.0,-38280.0,253751.0,-1.46,-15.40,120.69,86.70,129.0,-0.15,1.54,39.09,XY24,NTT,BANKS
45,INDIAMART,2327.09,4850.92,-48.26,H-SC,78.12,134763.0,11427.0,122338.0,-1.45,9.27,90.78,108.45,122.0,0.09,0.99,33.69,AR,ATH,MISC
4,ALKYLAMINE,2347.98,4546.37,6.93,X-SC,97.92,99489.0,-1474.0,96007.0,-1.17,-1.46,96.50,93.63,64.0,-0.02,0.73,48.19,SR,ATH,CHEMICALS
74,SIEMENS,4406.90,7969.85,-2.86,X-LC,26.04,82200.0,-27972.0,117045.0,-0.72,-25.39,142.39,80.85,14.0,-0.24,0.60,21.07,AR,ATH,ELECTRICAL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,93.75,132408.0,-10071.0,10699.0,-0.16,-7.07,8.08,0.44,243.0,-0.94,0.97,56.28,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.32,M-SC,11.46,148606.0,-22535.0,22484.0,0.46,-13.17,15.13,-0.03,189.0,-1.00,1.09,6.81,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.68,H-SC,88.54,149172.0,-25387.0,25419.0,0.14,-14.54,17.04,0.02,127.0,-1.00,1.09,29.35,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-212.00,H-SC,95.83,80808.0,-14508.0,14545.0,0.33,-15.22,18.00,0.04,154.0,-1.00,0.59,59.38,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-17.14,X-MC,15.62,86954.0,-15698.0,15695.0,-0.06,-15.29,18.05,-0.00,58.0,-1.00,0.64,32.18,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,12.50,49150.0,7895.0,2910.0,5.10,19.14,5.92,26.19,1.0,2.71,0.36,15.72,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.38,X-LC,36.46,265941.0,-29993.0,123902.0,0.26,-10.14,46.59,31.73,1.0,-0.24,1.94,6.45,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.55,X-LC,61.46,290087.0,28486.0,117137.0,0.12,10.89,40.38,55.67,2.0,0.24,2.12,17.66,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-16.87,X-LC,27.08,260288.0,-10059.0,44691.0,0.46,-3.72,17.17,12.81,6.0,-0.23,1.90,8.86,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-12.22,X-LC,29.17,278168.0,12742.0,44173.0,-0.04,4.80,15.88,21.44,9.0,0.29,2.03,11.76,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WHIRLPOOL,1167.49,2270.0,-37.55,M-SC,89.58,206020.0,34399.0,127671.0,-0.01,20.04,61.97,94.43,198.0,0.27,1.51,52.06,XR,NTT,DURABLES
88,UNIONBANK,123.87,163.0,0.15,M-LC,30.21,173495.0,32655.0,11832.0,0.66,23.19,6.82,31.59,89.0,2.76,1.27,50.71,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,1.67,M-LC,34.38,101970.0,23608.0,10421.0,0.36,30.13,10.22,43.43,77.0,2.27,0.75,37.05,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,70.83,72822.0,38373.0,1573.0,-0.32,111.39,2.16,115.97,NaN,24.39,0.53,11.36,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,95.22,M-MC,52.08,233916.0,8954.0,158689.0,-0.48,3.98,67.84,74.53,185.0,0.06,1.71,35.81,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,26.30,H-SC,45.83,129163.0,3866.0,55243.0,-0.14,3.09,42.77,47.18,139.0,0.07,0.94,61.59,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.09,H-SC,50.00,119148.0,17144.0,27881.0,-0.07,16.81,23.40,44.14,124.0,0.61,0.87,35.78,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,1.67,M-LC,34.38,101970.0,23608.0,10421.0,0.36,30.13,10.22,43.43,77.0,2.27,0.75,37.05,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,30.21,173495.0,32655.0,11832.0,0.66,23.19,6.82,31.59,89.0,2.76,1.27,50.71,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,70.83,72822.0,38373.0,1573.0,-0.32,111.39,2.16,115.97,NaN,24.39,0.53,11.36,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.55,M-SC,89.58,206020.0,34399.0,127671.0,-0.01,20.04,61.97,94.43,198.0,0.27,1.51,52.06,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,84.38,166904.0,15959.0,93500.0,0.81,10.57,56.02,72.51,69.0,0.17,1.22,14.97,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-48.26,H-SC,78.12,134763.0,11427.0,122338.0,-1.45,9.27,90.78,108.45,122.0,0.09,0.99,33.69,AR,ATH,MISC
54,KPIGREEN,497.21,731.79,26.30,H-SC,45.83,129163.0,3866.0,55243.0,-0.14,3.09,42.77,47.18,139.0,0.07,0.94,61.59,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,7.29,17568.0,-32025.0,949.0,1.83,-64.58,5.40,-62.66,NaN,-33.75,0.13,17.81,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,67853.0,-45696.0,85746.0,0.74,-40.24,126.37,35.27,261.0,-0.53,0.50,66.07,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.00,7266.67,M-SC,90.62,80848.0,-12962.0,89580.0,1.82,-13.82,110.80,81.67,242.0,-0.14,0.59,56.56,XR,NTT,CERAMICS
58,MASFIN,326.60,399.50,-18.25,H-SC,76.04,92655.0,-5325.0,27194.0,0.47,-5.43,29.35,22.32,137.0,-0.20,0.68,34.38,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.93,X-SC,97.92,99489.0,-1474.0,96007.0,-1.17,-1.46,96.50,93.63,64.0,-0.02,0.73,48.19,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-28.73,L-SC,100.00,143281.0,1439.0,55135.0,1.34,1.01,38.48,39.88,259.0,0.03,1.05,45.20,XY24,NTT,POWER
15,BLUESTARCO,1646.70,2337.55,-0.67,H-SC,98.96,186440.0,21770.0,47318.0,1.20,13.22,25.38,41.95,121.0,0.46,1.36,21.67,X40N,ATH,AC
4,ALKYLAMINE,2347.98,4546.37,6.93,X-SC,97.92,99489.0,-1474.0,96007.0,-1.17,-1.46,96.50,93.63,64.0,-0.02,0.73,48.19,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,67853.0,-45696.0,85746.0,0.74,-40.24,126.37,35.27,261.0,-0.53,0.50,66.07,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-212.00,H-SC,95.83,80808.0,-14508.0,14545.0,0.33,-15.22,18.00,0.04,154.0,-1.00,0.59,59.38,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.87
1,25,41.91
2,50,70.47


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.48
LC    33.87
MC    21.63
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.53
X40      12.21
XY25     11.96
X40N     11.95
XR       10.15
OX40N     8.24
AR        8.19
MH        1.69
X5K       1.46
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.65
M-SC    15.97
H-LC    14.10
X-LC    12.10
H-MC     9.56
X-MC     7.62
M-LC     6.63
X-SC     4.40
M-MC     4.09
L-SC     2.46
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.05,-1.61,35.61
IT,12.09,-9.82,63.74
FINANCE,7.87,-12.10,52.59
BANKS,7.85,-6.34,49.73
MISC,6.62,-14.95,68.17
PAINTS,5.72,-11.52,33.50
ELECTRICAL,5.52,2.41,43.18
HEALTHCARE,4.57,-3.14,33.27
AUTO,4.33,-13.98,64.09


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2689994.0,27
XR,1072719.0,13
AR,1022716.0,10
X40,646546.0,9
X40N,528010.0,11
XY25,481369.0,8
OX40N,304616.0,11
SR,177300.0,2
MH,84042.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1957653.0,20
M-SC,1757995.0,21
H-LC,609543.0,12
H-MC,525985.0,8
X-LC,518332.0,7
X-MC,410853.0,6
M-MC,410623.0,3
X-SC,317963.0,5
M-LC,266715.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      869123.0      6
M-SC       XY24      812036.0      7
H-SC       AR        522534.0      3
M-SC       XR        329979.0      4
M-MC       XY24      299953.0      2
H-MC       XY24      296535.0      3
X-LC       X40       241039.0      2
H-SC       XR        213981.0      3
           X40N      208251.0      4
H-LC       X40N      194347.0      4
M-SC       AR        182353.0      2
           OX40N     160464.0      5
M-LC       XY24      158474.0      3
H-LC       X40       155748.0      3
L-SC       XR        147549.0      2
X-MC       X40       146524.0      2
X-SC       XR        123346.0      1
M-SC       XY25      120627.0      1
X-LC       XY25      117281.0      3
           AR        117045.0      1
M-MC       XR        110670.0      1
X-MC       XY25      102972.0      1
X-SC       SR         96007.0      1
M-LC       XR         93500.0      1
X-MC       XY24       93336.0      1
H-SC       OX40N      88521.0      3
M-SC       SR         81293.0      1
H-LC       AR         79917.0      1
H-MC       XY25       79150.0      1
M-SC       X40        71243.0      1
H-LC       X200       68743.0      1
           X5K        67919.0      2
L-SC       AR         59904.0      1
H-MC       AR         59390.0      1
X-SC       XY24       58674.0      1
H-SC       MH         55243.0      1
L-SC       XY24       55135.0      1
L-MC       XR         53694.0      1
X-MC       X40N       52326.0      1
L-LC       XY25       46598.0      1
X-LC       X40N       42967.0      1
H-LC       XY24       42869.0      1
X-SC       OX40N      39936.0      2
H-MC       X40        31992.0      1
           X40N       30119.0      1
           MH         28799.0      1
X-MC       OX40N      15695.0      1
M-LC       XY25       14741.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
